In [ ]:
cc.VerficiationHandler.close_browser()

## Time to walk the elements of the "Bertha" mother element
### This has turned out to be a big thing.  It needs to be tackled for sucessful parsing of nathan and tom elements.  
### We do not look for indepentent trees for verified pointers.  We exploit the fact that the nathans and toms are paired by the nature of the verified pointer
### If I have a nathan and I walk a portion of the tree through, the mother element to get to tom, then I should be able to do the same thing for other elements with the same name as the parent element of nathan that are decendants of bertha.


In [1]:
% run contactsScraper.py

Done
Done
KEYS
Done
Done
RECORDS COLLECTED
DATAFRAMES READY
RECORD OUTPUT READY
CONTACT CHECKER READY


In [5]:
import numpy as np
import pandas as pd

In [2]:
orgsForToday = ['National Association for Multi-Ethnicity In Communications (NAMIC)',
                'Association for Women in Science',
                'Brain Injury Association of America',
                'American Society of Home Inspectors',
                'NAADAC, the Association for Addiction Professionals',
                'American Public Transportation Association',
                'Indiana Soybean Alliance',
                'Associated Builders and Contractors (ABC)',
                'National Association of Social Workers',
                'American Marketing Association (AMA)']

In [3]:
org  = orgsForToday[8]

vh = cc.MotherSetVerifier(org)

No Fred
No Fred


In [4]:
cc.VerificationHandler.orgRecords.orgSessionStatusCheck()

['Tue Oct 17, 2017  12:44:08', '4.917', 's', 'Good']

In [6]:
pointers = vh.verifiedPointers

In [7]:
len(pointers)

2